# 2.6.用插件扩展答疑机器人的能力边界
## 🚄 前言
在前面课程中，你已经掌握了通过优化提示词和检索流程提高答疑机器人效果的方法。但目前的答疑机器人依然有一定的局限性，本章节将带你发掘这些不足并引入智能体（Agent）应用以解决这些问题，它以大模型为基础，同时可以拓展大模型的能力，就像给大脑配备了四肢。

## 🍁 课程目标
学完本节课程后，你将能够：

* 掌握Agent系统的核心理念
* 熟悉Multi-Agent系统的设计与实现
* 掌握核心工具与框架以应对复杂任务

## 1.机器人的局限性与解决方案
一些同事希望答疑机器人能具备这样一种功能：只需说出“帮我请明天的假”，机器人便能自动提交请假申请单。然而，传统的答疑机器人有其固有局限性：

In [ ]:
# 导入所需的依赖包
from config.load_key import load_key
import os
# 加载API密钥
load_key()
# 生产环境中请勿将 API Key 输出到日志中，避免泄露
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

In [ ]:
from fontTools.ttLib.tables.ttProgram import instructions

from chatbot import rag
# 上一章节已经建立了索引，因此这里可以直接加载索引。如果需要重建索引，可以增加一行代码：rag.indexing()
index = rag.load_index()
query_engine = rag.create_query_engine(index=index)

rag.ask('张三的HR明天想请个假', query_engine=query_engine)

从上面的示例中你会发现当前大模型只是文本输入输出的问答系统无法和外界交互。

<img src="https://img.alicdn.com/imgextra/i3/O1CN01rdstgY1uiZWt8gqSL_!!6000000006071-0-tps-1970-356.jpg" alt="image" width="600" hight="300">


为了解决这个问题，你可以为答疑机器人引入一种新能力：动态解析用户需求并采取相应行动。例如，为了让问答机器人能够帮助用户请假，你需要让大模型解析用户的需求，并调用相应的API（如请假API）。这正是智能体应用的核心思想——通过任务分解和自动化执行，智能体能够高效地响应并完成复杂的操作。

<img src="https://gw.alicdn.com/imgextra/i4/O1CN01Xud4YB1D1UniMmUaN_!!6000000000156-0-tps-952-410.jpg" alt="image" width="600" hight="700">

## 2.如何构建 Agent

通常来说构建一个智能体分为以下步骤，你将如下图所示一步步完成一个智能体的构建。

<img src="https://img.alicdn.com/imgextra/i3/O1CN01wyaLNO1VE0hqPKAIm_!!6000000002620-0-tps-1083-91.jpg" alt="image" width="600">

### 2.1明确目标
“不谋全局者，不足谋一域。” —— 在任何复杂任务中，明确目标都是迈向成功的第一步。正如绘制一张地图需要先确定目的地，在构建答疑机器人时，你也需要先清晰地定义任务的核心目标。

你想要答疑机器人能够从公司的私有数据库中查询员工信息，并能够帮助用户完成请假申请的同时在数据库中进行记录与更新。

所以你的第一个目标是：将用户所有对公司内人员信息类型的提问转化为数据库查询的工具函数。 具体而言，这包括：

- 将用户输入的自然语言问题转化为对应的 SQL 查询语句（即 NL2SQL，自然语言转 SQL）。
- 使用生成的 SQL 查询语句访问数据库，获取对应的查询结果。
- 将查询结果作为工具函数的输出，最终返回给用户。



### 2.2定义工具函数
接下来，在配置好环境变量后你就可以开始搭建一个Agent智能体了。

当然，从零来构建一个Agent需要处理复杂的底层实现，这往往需要大量的时间和精力，因此你可以使用 Assistant API 来帮助你更高效地构建 Agent。

Assistant API 是一种简化智能体应用创建过程的接口。它提供了丰富的功能，包括支持多种基础模型、灵活的工具调用、对话管理和高度可扩展性。

通过 Assistant API，你可以专注于智能体的核心功能，而不需要处理繁琐的底层实现。

首先你需要定义一些工具函数，假设你的答疑机器人需要有从数据库查询员工信息的功能。

为了帮助你更多地关注到Agent的内容，你需要模拟一个查询步骤，而不实际去数据库中查询。

> 假设员工表名为employee，字段包括department（部门）、name（姓名）、HR。

> 如果你希望继续提升大模型NL2SQL的性能，请前往微调教程学习。

In [ ]:
# 导入依赖
from llama_index.llms.dashscope import DashScope
from llama_index.core.base.llms.types import MessageRole, ChatMessage
import re # 导入正则表达式库，用于处理空格

def normalize_sql(sql_string):
    '''
    对SQL语句进行标准化处理，以方便比较。
    '''
    # 1. 转换为小写
    s = sql_string.lower()
    # 2. 移除首尾空格
    s = s.strip()
    # 3. 移除末尾的分号
    if s.endswith(';'):
        s = s[:-1]
    # 4. (可选但推荐) 将多个空格替换为单个空格
    s = re.sub(r'\s+', ' ', s)
    return s

# 定义一个员工查询函数
def query_employee_info(query):
    '''
    输入用户提问，输出员工信息查询结果
    '''
    # 1. 首先根据用户提问，使用NL2SQL生成SQL语句
    llm = DashScope(model_name="qwen-plus")
    messages = [
        ChatMessage(role=MessageRole.SYSTEM, content='''你有一个表叫employees，记录公司的员工信息，这个表有department（某部门）、name（姓名）、HR三个字段。
    你需要根据用户输入生成sql语句进行查询，只生成sql语句，不需要生成sql语句之外的内容，也不要把```sql```这个标签加上。'''),
        ChatMessage(role=MessageRole.USER, content=query)
    ]
    SQL_output_raw = llm.chat(messages).message.content
    
    # 对大模型输出的SQL进行标准化
    SQL_output_normalized = normalize_sql(SQL_output_raw)
    
    # 打印出原始和标准化后的SQL语句
    print(f'原始SQL语句为：{SQL_output_raw}')
    print(f'标准化后SQL语句为：{SQL_output_normalized}')
    
    # 2. 根据标准化后的SQL语句去查询数据库（此处为模拟查询），并返回结果
    # 注意：我们的比较目标也应该是标准化的字符串
    if SQL_output_normalized == "select count(*) from employees where department = '教育部门'":
        return "教育部门共有66名员工。"
    if SQL_output_normalized == "select hr from employees where name = '张三'":
        return "张三的HR是李四。"
    if SQL_output_normalized == "select department from employees where name = '王五'":
        return "王五的部门是后勤部。"
    else:
        return "抱歉，我暂时无法回答您的问题。"

# 测试一下这个函数
result = query_employee_info("教育部门有几个人")
print(f"查询结果: {result}")

result = query_employee_info("张三的HR是谁")
print(f"查询结果: {result}")


### 2.3将工具函数与大模型集成进Agent中
你已经定义好了工具函数，接下来就要将它们与大模型通过Assistant API集成到Agent中。

通过Assistants.create方法，你可以创建一个新的Agent，并通过model（模型名称）、name（Agent命名）、description（Agent的描述信息）、instructions（Agent非常重要的参数，用于提示Agent所具有的工具函数能力，同时也可以规范输出格式）、tools（工具函数通过该参数传入）参数来定义Agent。

> 其中，tools参数中的function.name用于指定工具函数，但需要为字符串格式，因此可以通过一个map方法映射到工具函数上。

In [ ]:
# 引入依赖
from dashscope import Assistants, Messages, Runs, Threads
import json

# 定义公司小蜜
ChatAssistant = Assistants.create(
    # 在此指定模型名称
    model="qwen-plus",
    # 在此指定Agent名称
    name='公司小蜜',
    # 在此指定Agent的描述信息
    description='一个智能助手，能够查询员工信息，帮助员工发送请假申请，或者查询公司规章制度。',
    # 用于提示大模型所具有的工具函数能力，也可以规范输出格式
    instructions='''你是公司小蜜，你的功能有以下三个：
    1. 查询员工信息。例如：查询员工张三的HR是谁；
    2. 发送请假申请。例如：当员工提出要请假时，你可以在系统里帮他完成请假申请的发送；
    3. 查询公司规章制度。例如：我们公司项目管理的工具是什么？
    请准确判断需要调用哪个工具，并礼貌回答用户的提问。
    ''',
    # 将工具函数传入
    tools=[
        {
            # 定义工具函数类型，一般设置为function即可
            'type': 'function',
            'function': {
                # 定义工具函数名称，通过map方法映射到query_employee_info函数
                'name': '查询员工信息',
                # 定义工具函数的描述信息，Agent主要根据description来判断是否需要调用该工具函数
                'description': '当需要查询员工信息时非常有用，比如查询员工张三的HR是谁，查询教育部门总人数等。',
                # 定义工具函数的参数
                'parameters': {
                    'type': 'object',
                    'properties': {
                        # 将用户的提问作为输入参数
                        'query': {
                            'type': 'str',
                            # 对输入参数的描述
                            'description': '用户的提问。'
                        },
                    },
                    # 在此声明该工具函数需要哪些必填参数
                    'required': ['query']},
            }
        }
    ]
)
print(f'{ChatAssistant.name}创建完成')
# 建立Agent Function name与工具函数的映射关系
function_mapper = {
    "查询员工信息": query_employee_info
}
print('工具函数与function.name映射关系建立完成')

同时，你还可以封装一个辅助函数`get_agent_response`。

这段代码的功能是：当用户向智能体发出请求时，智能体通过 get_agent_response() 发送请求并获取响应。如果任务需要调用外部工具（如数据库查询），则智能体会根据工具函数的映射执行相应的操作，并将结果返回给用户。这使得智能体能够处理更复杂的任务，而不仅仅是简单的问答。

通过Assistant API获得Agent回复的过程需要涉及到如thread、message、run等概念，如果你对这些概念与细节感兴趣，请参考[阿里云Assistant API官方文档](https://help.aliyun.com/zh/model-studio/developer-reference/assistantapi/)。

> 如果你希望给Agent配备更多的能力，可以添加工具函数，并在function_mapper与tools中建立映射关系。

In [ ]:
# 输入message信息，输出为指定Agent的回复
def get_agent_response(assistant, message=''):
    # 创建一个新的会话线程
    thread = Threads.create()
    
    # 创建一条消息并发送到该会话线程
    message = Messages.create(thread.id, content=message)
    
    # 创建一个运行实例（运行请求），将会话线程与Assistant（智能体）关联起来
    run = Runs.create(thread.id, assistant_id=assistant.id)
    
    # 等待运行完成，检查任务是否完成
    run_status = Runs.wait(run.id, thread_id=thread.id)
    
    # 如果任务运行失败，则输出错误信息
    if run_status.status == 'failed':
        print('run failed:')
        
    # 如果需要工具来辅助模型进行操作（如查询数据库、发送请求等）
    if run_status.required_action:
        # 获取需要调用的工具函数的详细信息
        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
        
        # 获取工具函数的名称（function name）
        func_name = f['name']
        
        # 获取调用工具函数时需要的输入参数
        param = json.loads(f['arguments'])
        
        # 打印出工具的名称和参数信息
        print("function is", f)
        
        # 根据工具函数的名称，通过一个映射（function_mapper）找到对应的实际工具函数
        # 这里使用了一个字典映射（function_mapper），它将工具函数名称与具体的函数对应
        if func_name in function_mapper:
            # 使用映射找到实际工具函数并传递参数，获取结果
            output = function_mapper[func_name](**param)
        else:
            # 如果找不到对应的函数，输出为空
            output = ""
        
        # 将工具函数的输出（结果）准备提交
        tool_outputs = [{
            'output': output
        }]
        
        # 提交工具的输出结果回给运行实例
        run = Runs.submit_tool_outputs(run.id, thread_id=thread.id, tool_outputs=tool_outputs)
        
        # 等待运行完成
        run_status = Runs.wait(run.id, thread_id=thread.id)
    
    # 获取最终的运行结果
    run_status = Runs.get(run.id, thread_id=thread.id)
    
    # 获取消息记录列表
    msgs = Messages.list(thread.id)
    
    # 返回Agent的回复内容
    return msgs['data'][0]['content'][0]['text']['value']


### 2.4尝试对话
你已经完成了一个简单的单Agent系统构建，在正式投入使用之前测试是必不可少的一环，你可以尝试与答疑机器人进行对话：

In [ ]:
query_stk = [
    "谁是张三的HR？",
    "教育部门一共有多少员工？",
    "王五在哪个部门？",
]
for query in query_stk:
    print("提问是：")
    print(query)
    print("思考过程与最终输出是：")
    print(get_agent_response(ChatAssistant,query))
    print("\n")


从测试结果可以看出，拓展了功能之后的答疑机器人达到了你预期的效果。


<img src="https://img.alicdn.com/imgextra/i4/O1CN01q6NyMr237JtdJFO8Q_!!6000000007208-2-tps-2082-974.png" alt="image" width="600" height="300">

在实际应用中，智能体不仅可以与外界交互，还能通过不同的模块化设计来增强其处理复杂任务的能力。智能体的工作原理可以从以下几个核心模块进行理解：

- 工具模块

    工具模块负责定义和管理智能体能够使用的各种工具。包括工具的描述、参数以及功能特性。这一模块确保智能体能够理解并有效使用这些工具来完成任务。

- 记忆模块

    记忆模块可以分为长期记忆和短期记忆。
  
    长期记忆用于存储持久的信息和经验，帮助智能体进行模式学习、知识积累和个性化服务。
    
    短期记忆则用于临时存储当前任务相关的信息，以支持智能体在任务执行过程中实时调整决策。
- 计划能力

    计划能力模块负责任务的规划。通过智能体的决策能力，这部分帮助智能体分解复杂任务，制定具体的行动步骤和策略，确保任务顺利完成。

- 行动能力

    行动能力与工具模块紧密配合，确保智能体能够选择合适的工具，并通过容器执行相应的操作。行动能力是智能体实现任务的核心，确保其能够根据既定计划和决策，有效地实施各项任务。

通过这些模块的协作，智能体能够处理复杂任务，提升任务执行的效率和精准度，突破传统方法的局限。



如果你对这些概念感兴趣，请参考[阿里云大模型ACA课程](https://edu.aliyun.com/course/3126500/lesson/342570389?spm=a2cwt.28196072.ACA13.12.660e6e7b6G8Ye7)与本章节拓展阅读部分。

## 3. 使用大模型进行意图识别

在上一节中，你已经成功构建了一个具备使用工具能力的初步智能体（Agent），这比单纯的 RAG 问答机器人又进了一大步。然而，当前的智能体仍然很简单，它只会假定用户的每一个问题都应该由它所拥有的工具（查询员工信息）来处理。如果用户提出了一个完全不同类型的请求，比如，让机器人帮忙检查一句话有没有语病，这时会发生什么呢？

试着做一个直观的实验。假设一位同事想让你开发的机器人帮忙检查一下文档里的某句话有没有语病：

In [ ]:
from chatbot import rag
# 看看当机器人遇到一个非知识问答类请求时会发生什么
rag.ask('请帮我审查下这句话：技术内容工程师需要设计和开发⾼质量的教育教材和课程吗？', query_engine=query_engine)

>**预期输出（示例）**
>
>是的，根据提供的上下文信息，技术内容工程师的核心职责之一就是设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件和设计评估工具等，以确保内容符合教育标准和学习目标。

看到这个结果，你可能会感到困惑。机器人并没有“审查”这句话，而是直接“回答”了这个问题。它为什么会犯这样的错误？

还记得 RAG 工作流程吗？机器人会先从知识库里检索与问题相关的内容。在这个例子中，它找到了以下信息：


> 片段: 内容开发工程师
>......
>1. 内容研究与分析
>对最新的教育技术趋势、学习理论和市场需求进行深入研究。这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中。通过持续的市场调研，我能够确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求。
>2. 教材和课程开发
>根据研究和市场反馈，我将设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件、设计评估工具等。我的职责还包括确保内容符合教育标准和学习目标，以提供全面的学习体验。同时，我会考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。
>......


现在，问题清晰了。RAG 流程忠实地完成了它的任务——它在你的问题中看到了“技术内容工程师”这个关键词，并成功地从知识库中检索到了相关的职责描述。然后，大模型基于这些信息，给出了一个内容正确、但完全“答非所问”的答案。

怎么解决这个问题呢？

你可以退一步思考。如果你是这个机器人，你会怎么做？你可能会先判断一下：“用户这次是想让我**回答问题**，还是想让我**检查语法**？”

这个“判断”的动作，就是解决问题的关键。你可以让大模型自己来做这个判断！

试试这个最直观的思路：在处理用户请求之前，先用一个简单的提示词让大模型判断用户的**意图**。

In [ ]:
from chatbot import rag, llm
# 引入一个“意图判断”步骤
question = "请帮我审查下这句话：技术内容工程师需要设计和开发⾼质量的教育教材和课程吗？"

# 第一步：判断用户意图
# 用一个简单的提示词让大模型做一个二选一的判断
intent_prompt = f"""
用户的请求是关于“知识库问答”还是“文本审查”？请只回答类别名称。
用户请求：{question}
"""
intent = llm.invoke(intent_prompt)
print(f"识别到的用户意图是：{intent}")

# 第二步：根据意图选择不同的处理方式

# 问题属于文档审查，不使用RAG
if "文本审查" in intent:
    print("意图是文本审查，将不使用 RAG，直接调用大模型进行审查...")
    llm.invoke_with_stream_log(question)
else:
    print("意图是知识库问答，将使用 RAG...")
    rag.ask(question, query_engine)

>**预期输出（示例）**
>
>识别到的用户意图是：文本审查
>
>意图是文本审查，将不使用 RAG，直接调用大模型进行审查...
>
>这句话基本上是正确的，但可以稍微调整一下，使其更加流畅和准确。你可以这样表达：
>
>“技术内容工程师是否需要设计和开发高质量的教育教材和课程？”

通过这个简单的代码实验，你亲手实现了一个更智能的工作流程。这里并没有引入什么复杂的技术，只是在流程中增加了一个判断步骤，就让机器人学会了“看情况办事”。

这个**识别用户真实目的**的过程，在行业内有一个专门的术语，叫做 **意图识别（Intent Recognition）**。

你可以把意图识别想象成一个**智能客服前台**。当客户走近时，前台不会立刻把公司手册丢给他，而是会先问：“您好，请问有什么可以帮您？” 根据客户的回答（意图），前台再决定是引导他去编辑部、销售部，还是仅仅回答一个简单的问题。

<img src="https://img.alicdn.com/imgextra/i1/O1CN0126zKe71PAuJjWfQ3N_!!6000000001801-0-tps-2254-1080.jpg" width="800">

这正是 2.1 章提到的 **上下文工程（Context Engineering）** 的一个更深层次的应用。回顾一下：

*   **通过 RAG 添加知识**，是你为大模型**填充上下文**，解决它“不知道”的问题。
*   **通过意图识别**，你开始**控制上下文**，决定在何时、何种情况下、填充什么样的上下文（甚至不填充）。

你正在从一个单纯的“信息投喂员”，转变为一个能够设计和指挥复杂工作流的“总工程师”。通过为不同的意图设计不同的处理流程，你可以让你的应用更高效、更节省成本，并且极大地减少因信息干扰而导致的“答非所问”。

在接下来的内容中，你将了解系统化的思路，构建一个更强大的意图识别路由器，让你的答疑机器人能够处理更多样的任务。

### 3.1 意图识别

接下来，将构建提示词使大模型对问题分类。由于经过意图识别后要取得格式化的内容，才能进行文档审查或者使用RAG应用，所以为了能将用户的问题准确分类，将考虑以下提示词技巧：
- 明确输出格式：指定输出格式，使分类结果规范且易于解析。
- Few-shot 示例：提供示例，帮助大模型理解每个类别的特征和分类规则。

In [ ]:
from chatbot import llm
  
# 构建提示词
prompt = '''
【角色背景】
你是一个问题分类路由器，负责判断用户问题的类型，并将其归入下列三类之一：
1. 公司内部文档查询
2. 内容翻译
3. 文档审查

【任务要求】
你的任务是根据用户的输入内容，判断其意图并仅选择一个最贴切的分类。请仅输出分类名称，不需要多余的解释。判断依据如下：

- 如果问题涉及公司政策、流程、内部工具或职位描述与职责等内容，选择“公司内部文档查询”。
- 如果问题涉及任意一门非中文的语言，且输入中出现任何外语或“翻译”等字眼，选择“内容翻译”。
- 如果问题涉及检查或总结外部文档或链接内容，选择“文档审查”。
- 用户的前后输入与问题分类并没有任何关系，请单独为每次对话考虑分类类别。

【Few-shot 示例】
示例1：用户输入：“我们公司内部有哪些常用的项目管理工具？”
分类：公司内部文档查询

示例2：用户输入：“请翻译下列句子：How can we finish the assignment on time?”
分类：内容翻译

示例3：用户输入：“请审查下这个链接下的文档：https://help.aliyun.com/zh/model-studio/user-guide/long-context-qwen-long”
分类：文档审查

示例4：用户输入：“请审查以下内容：技术内容工程师需要设计和开发⾼质量的教育教材和课程吗？”
分类：文档审查

示例5：用户输入：“技术内容工程师核心职责是什么？”
分类：公司内部文档查询

【用户输入】
以下是用户的输入，请判断分类：
'''

# 获取问题的类型
def get_question_type(question):
    return llm.invoke(prompt + question)

print(get_question_type('https://www.promptingguide.ai/zh/techniques/fewshot'),'\n')
print(get_question_type('That is a big one I dont know why'),'\n')
print(get_question_type('作为技术内容工程师有什么需要注意的吗？'),'\n')

通过明确的输出格式和 few-shot 示例，答疑机器人可以更准确地识别问题类型并输出符合预期的格式。这种优化让分类任务更加标准化，为接下来添加意图识别到答疑机器人中打下了基础。


### 3.2 将意图识别应用到答疑机器人中

对用户的问题进行意图识别后，你就可以让答疑机器人先识别问题的类型，再使用不同的提示词和工作流程来回答问题。

In [ ]:
def ask_llm_route(question):
    question_type = get_question_type(question)
    print(f'问题：{question}\n类型：{question_type}')
  
    reviewer_prompt = """
    【角色背景】
    你是文档纠错专家，负责找出文档中或网页内容的明显错误
    【任务要求】
    - 你需要言简意赅的回复。
    - 如果没有明显问题，请直接回复没有问题\n
    【输入如下】\n"""
  
    translator_prompt = """
   【任务要求】
    你是一名翻译专家，你要识别不同语言的文本，并翻译为中文。
    【输入如下】\n"""

    if question_type == '文档审查':
        return llm.invoke(reviewer_prompt + question)
    elif question_type == '公司内部文档查询':
        return rag.ask(question, query_engine=query_engine)
    elif question_type == '内容翻译':
        return llm.invoke(translator_prompt + question)
    else:
        return "未能识别问题类型，请重新输入。"

query_engine =rag.create_query_engine(index=rag.load_index())

In [ ]:
# 问题1
print(ask_llm_route('https://www.promptingguide.ai/zh/techniques/fewshot'),'\n')

# 问题2
print(ask_llm_route('请帮我检查下这段文档：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？'),'\n')

# 问题3
print(ask_llm_route('技术内容工程师有需要进行内容优化与更新与跨部门合作吗？'),'\n')

# 问题4:
print(ask_llm_route('A true master always carries the heart of a student.'),'\n')


从上述实验中可以看出，通过引入意图识别这一步骤，我们的答疑机器人变得更加智能。这正是**上下文工程**中“控制流”设计的体现。它不再是一个简单的“提问-检索-回答”的线性流程，而是根据任务类型动态地调整其行为。这样做的好处是显而易见的：
- 节省资源：对于检查文档错误的问题，大模型其实可以直接回复，并不需要检索参考资料，之前的实现存在资源浪费。
- 避免误解：之前的实现每次会检索参考资料，这些被召回的相关文本段可能会干扰大模型理解问题，导致答非所问。

我们刚刚构建的意图识别模块，本质上是一个简单的“路由器”，它决定了用户的请求应该走哪条处理路径。这种路由和规划的思想，是更高级智能体系统的核心。现在你已经了解了如何应对不同类型的任务，让我们回到最初的工具型智能体，看看如何处理需要**组合使用多个工具**的更复杂的任务。这将引导我们进入多智能体（Multi-Agent）系统的世界。

## 4.多智能体Multi-Agent
当完成员工信息的查询后，接下来你还需要对员工请假申请进行申请与记录，所以你需要再加一个新的工具函数以满足此需求。

这个工具函数将员工输入的请假日期作为输入参数，并返回一个申请成功的字符串。为了帮助你更多地关注到Agent的内容，下方的示例模拟了请假申请步骤，而没有实际去公司系统中提交请假申请。

In [ ]:
def send_leave_application(date):
    '''
    输入请假时间，输出请假申请发送结果
    '''
    return f'已为你发送请假申请，请假日期是{date}。'

# 测试一下这个函数
print(send_leave_application("明后两天"))

在确定新的工具函数正常工作后，你需要将这个新函数集成进你之前创建的agent中：

In [ ]:
new_tool = {'type': 'function',
            'function': {
                'name': '发送请假申请',
                'description': '当需要帮助员工发送请假申请时非常有用。',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        # 需要请假的时间
                        'date': {
                            'type': 'str',
                            'description': '员工想要请假的时间。'
                        },
                    },
                    'required': ['date']},
            }
           }
ChatAssistant.tools.append(new_tool)
function_mapper["发送请假申请"] = send_leave_application
print('请假工具函数与function.name映射关系建立完成')

在确认集成成功后，你可以来测试一下模型的输出效果以确保一切功能正常运作：

In [ ]:
get_agent_response(ChatAssistant,"张三的HR是谁？给他请三天假")

通过上面的输出结果，你会发现在处理复杂任务时，特别是当机器人需要在一个请求中执行多个操作时，单个智能体可能无法有效完成所有子任务。

例如，用户请求“张三的HR是谁？给他请三天假”，这就涉及到员工信息查询和请假申请两个操作。单个智能体通常只能处理一种任务，无法同时调动多个工具或API接口来完成所有子任务。

为了克服这种多操作需求的局限性，你可以为答疑机器人引入一种新能力：将任务拆解成多个独立的模块进行处理，而多智能体系统正是为此而生。

多智能体系统通过将任务拆解成多个子任务，并由不同的智能体分别处理这些任务，从而克服了单一智能体无法同时完成多个操作的局限性。每个智能体专注于一个特定任务，像一个团队中的成员，各司其职，最终协作完成整个任务。

这种设计不仅能够提高任务处理的效率，还能增强灵活性，确保每个子任务得到专门的处理。

Multi-Agent系统有多种设计思路，本教程将介绍一个由Planner Agent、若干个负责执行工具函数的Agent，以及一个Summary Agent组成的Multi-Agent系统。
- Planner Agent规划智能体：
    根据用户的输入内容，选择要将任务分发给哪个Agent或Agent组合完成任务。
- 执行工具函数的Agent智能体：
    根据Planner Agent分发的任务，执行属于自己的工具函数。
- Summary Agent总结智能体：
    根据用户的输入，以及执行工具函数的Agent的输出，生成总结并返回给用户。
  
<a href="https://img.alicdn.com/imgextra/i1/O1CN01kX9Asm1dNdxmW18nG_!!6000000003724-2-tps-2074-935.png" target="_blank">
<img src="https://img.alicdn.com/imgextra/i1/O1CN01kX9Asm1dNdxmW18nG_!!6000000003724-2-tps-2074-935.png" alt="image" width="800" height="400">
</a>

回到之前的示例——“张三的HR是谁？给他请三天假”。在多智能体系统中，这个任务会被拆解成两个子任务：

查询张三的HR信息：由一个Agent负责。

发送请假申请：由另一个Agent负责。

通过多智能体系统，Planner Agent首先分析用户请求并拆解成这两个子任务，然后将每个任务交给对应的执行Agent处理。最后，Summary Agent会将各个Agent的结果汇总，生成最终的响应。

### 4.1 Planner Agent
Planner Agent 是 Multi-Agent 系统的核心部分，它负责分析问题，并决定将任务分发到哪个Agent或Agent组合上。

首先利用 Assistant API 创建 Planner Agent，此处你可以先不对instructions进行指定：

In [ ]:
# 决策级别的agent，决定使用哪些agent，以及它们的运行顺序。
planner_agent = Assistants.create(
    model="qwen-plus",
    name='流程编排机器人',
    description='你是团队的leader，你的手下有很多agent，你需要根据用户的输入，决定要以怎样的顺序去使用这些agent'
)

print("Planner Agent创建完成")

创建完成后你可以先来看看在未定义instructions时，Planner Agent 的输出是什么样的：

In [ ]:
print(get_agent_response(planner_agent,'谁是张三的HR？教育部门一共有多少员工？'))

从机器人的回答中你可以发现，其输出的回答包含许多额外的信息且没有告知所需要调用的Agent名称。

接下来你需要通过 instructions 指定其可调用的 Agent 及输出格式。

目前，你需要机器人能够帮助员工进行公司内部员工信息查询、请假申请以及其他日常对话。

另外，由于 Agent 的返回值为字符串格式，当你后续需要通过返回内容来分别调用所对应的智能体时会产生不便，所以你需要要求 Planner Agent 输出列表形式的字符串。例如：`["employee_info_agent", "leave_agent", "company_info_agent"]`，以便后续使用字符串解析工具将其转换为结构化数据。

In [ ]:
planner_agent=Assistants.update(planner_agent.id,instructions="""你的团队中有以下agent。
    employee_info_agent：可以查询公司的员工信息，如果提问中关于部门、HR等信息，则调用该agent；
    leave_agent：可以帮助员工发送请假申请，如果用户提出请假，则调用该agent；
    chat_agent：如果用户的问题无需以上agent，则调用该agent。

    你需要根据用户的问题，判断要以什么顺序使用这些agent，一个agent可以被多次调用。你的返回形式是一个列表，不能返回其它信息。比如：["employee_info_agent", "leave_agent"]或者["chat_agent"]，列表中的元素只能为上述的agent。""")

print("Planner Agent 的 instructions 已更新")

接下来尝试几个测试问题，看下Planner Agent能否分发到正确的Agent。

In [ ]:
query_stk = [
    "谁是张三的HR？教育部门一共有多少员工？",
    "王五在哪个部门？帮我提交下周三请假的申请",
    "你好"
]
for query in query_stk:
    print("提问是：")
    print(query)
    print(get_agent_response(planner_agent,query))
    print("\n")

对于这三个测试问题，Planner Agent都做出了正确的选择。

你可以观察到当Planner Agent返回任务规划结果后，其输出是一个描述任务执行顺序的列表形式字符串，例如：["employee_info_agent", "leave_agent"]。为了便于后续处理和执行，你需要将其转换为 Python 原生的列表结构（list）并保留相对应的调用顺序。在这里，你可以使用了 Python的ast.literal_eval方法，它可以安全地将字符串表达式解析为相应的 Python 数据类型，例如列表、字典等。

通过这种方式，你可以将任务规划结果转化为易于操作的列表对象，并逐步解析出每个任务的执行步骤，以简化后续的多智能体协作。

In [ ]:
import ast

# 使用Planner Agent获取任务规划
planner_response = get_agent_response(planner_agent, "王五在哪个部门？帮我提交下周三请假的申请")

# 将Planner Agent的字符串形式回复解析为列表
# Planner Agent返回的是一个描述调用顺序的列表形式字符串，例如：["employee_info_agent", "leave_agent"]
order_stk = ast.literal_eval(planner_response)

# 打印出Planner Agent的规划结果
print("Planner Agent的任务规划结果：")
for i, agent in enumerate(order_stk, start=1):
    print(f'第{i}步调用：{agent}')


### 4.2执行工具函数的Agent
上一章节你已经完成了Planner Agent的规划工作。它如同蚁巢中的蚁后，能够统筹规划任务并下达命令。然而，单靠蚁后是不足以让整个蚁巢运转起来的——需要无数的工蚁去执行具体任务，比如搜集食物或修筑巢穴。同样的道理，在你的多智能体系统中，仅有 Planner Agent 还不足以完成任务，必须为其配备执行具体任务的 工具函数Agent，才能真正实现整个系统的高效协作。

以下，你将需要基于上节中的规划结果，为两个不同任务创建独立的 执行工具函数Agent，使它们分别负责具体的操作任务。这种设计不仅让系统更加模块化，还能最大限度地发挥 Planner Agent 的协调作用。

> 需要确保agent变量名与Planner Agent的instructions中定义的agent变量名一致。

In [ ]:
# 员工信息查询agent
employee_info_agent = Assistants.create(
    model="qwen-plus",
    name='员工信息查询助手',
    description='一个智能助手，能够查询员工信息。',
    instructions='''你是员工信息查询助手，负责查询员工姓名、部门、HR等信息''',
    tools=[
        {
            'type': 'function',
            'function': {
                'name': '查询员工信息',
                'description': '当需要查询员工信息时非常有用，比如查询员工张三的HR是谁，查询教育部门总人数等。',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'query': {
                            'type': 'str',
                            'description': '用户的提问。'
                        },
                    },
                    'required': ['query']},
            }
        }
    ]
)
print(f'{employee_info_agent.name}创建完成')

# 请假申请agent
leave_agent = Assistants.create(
    model="qwen-plus",
    name='请假申请助手',
    description='一个智能助手，能够帮助员工提交请假申请。',
    instructions='''你是员工请假申请助手，负责帮助员工提交请假申请。''',
    tools=[
        {
            'type': 'function',
            'function': {
                'name': '发送请假申请',
                'description': '当需要帮助员工发送请假申请时非常有用。',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        # 需要请假的时间
                        'date': {
                            'type': 'str',
                            'description': '员工想要请假的时间。'
                        },
                    },
                    'required': ['date']},
            }
        }
    ]
)
print(f'{leave_agent.name}创建完成')
# 功能是回复日常问题。对于日常问题来说，可以使用价格较为低廉的模型作为agent的基座
chat_agent = Assistants.create(
    # 因为该Agent对大模型性能要求不高，因此使用成本较低的qwen-turbo模型
    model="qwen-turbo",
    name='回答日常问题的机器人',
    description='一个智能助手，解答用户的问题',
    instructions='请礼貌地回答用户的问题'
)
print(f'{chat_agent.name}创建完成')

### 4.3创建Summary Agent并测试Multi-Agent效果
在完成了Planner Agent和执行工具函数Agent的创建后，你还需要创建Summary Agent，该Agent会根据用户的问题与之前Agent输出的参考信息，全面、完整地回答用户问题。

In [ ]:
summary_agent = Assistants.create(
    model="qwen-plus",
    name='总结机器人',
    description='一个智能助手，根据用户的问题与参考信息，全面、完整地回答用户问题',
    instructions='你是一个智能助手，根据用户的问题与参考信息，全面、完整地回答用户问题'
)
print(f'{summary_agent.name}创建完成')

你可以将以上步骤封装为一个get_multi_agent_response函数，这样可以将复杂的多 Agent 协作过程抽象为一个简单接口。通过这种封装方式，用户只需提供输入问题，函数将：

1. 调用 Planner Agent，规划任务顺序。
2. 根据规划顺序依次调用对应的工具函数 Agent 执行任务。
3. 汇总所有任务结果，最后通过 Summary Agent 生成最终回答。

这种设计不仅让主流程更加清晰，还便于复用和扩展。

> 由于列表中的元素为字符串，因此通过一个agent_mapper方法将字符串格式的Agent映射到定义好的Agent对象。

In [ ]:
# 将列表中的字符串映射到Agent对象上
# 将字符串格式的Agent名称映射到具体Agent对象
agent_mapper = {
    "employee_info_agent": employee_info_agent,
    "leave_agent": leave_agent,
    "chat_agent": chat_agent
}

def get_multi_agent_response(query):
    # 获取Agent的运行顺序
    agent_order = get_agent_response(planner_agent,query)
    # 由于大模型输出可能不稳定，因此加入异常处理模块处理列表字符串解析失败的问题
    try:
        order_stk = ast.literal_eval(agent_order)
        print("Planner Agent正在工作：")
        for i in range(len(order_stk)):
            print(f'第{i+1}步调用：{order_stk[i]}')
        # 随着多Agent的加入，需要将Agent的输出添加到用户问题中，作为参考信息
        cur_query = query

        Agent_Message = ""
        # 依次运行Agent
        for i in range(len(order_stk)):
            cur_agent = agent_mapper[order_stk[i]]
            response = get_agent_response(cur_agent,cur_query)
            Agent_Message += f"*{order_stk[i]}*的回复为：{response}\n\n"
            # 如果当前Agent为最后一个Agent，则将其输出作为Multi Agent的输出
            if i == len(order_stk)-1:
                prompt = f"请参考已知的信息：{Agent_Message}，回答用户的问题：{query}。"
                multi_agent_response = get_agent_response(summary_agent,prompt)
                print(f'Multi-Agent回复为：{multi_agent_response}')
                return multi_agent_response
            # 如果当前Agent不是最后一个Agent，则将上一个Agent的输出response添加到下一轮的query中，作为参考信息
            else:
                # 在参考信息前后加上特殊标识符，可以防止大模型混淆参考信息与提问
                cur_query = f"你可以参考已知的信息：{response}你要完整地回答用户的问题。问题是：{query}。"
    # 兜底策略，如果上述程序运行失败，则直接调用大模型
    except Exception as e:
        return get_agent_response(chat_agent,query)

# 此处来用 “王五在哪个部门？帮我提交下周三请假的申请”进行一个测试
get_multi_agent_response("王五在哪个部门？帮我提交下周三请假的申请")

## 5.大模型平台的多智能体编排功能
上一小节你了解了Multi-Agent系统的设计理念与实现方法。可以看出，在自主构建一个Multi-Agent系统时，虽然能够提供高度的灵活性，但也伴随着一定的工作量和复杂性。对于许多企业来说，快速实现复杂业务逻辑更为重要。

[Dify](https://Dify.ai) 是智能体流程画布的开创者之一。Dify的画布工具让用户能够通过可视化的流程图来设计和管理智能体任务的执行逻辑，在业内树立了标杆。其直观的界面设计，为许多开发者提供了参考。

尽管 Dify.ai 在智能体流程编排方面表现出色，但相比之下，百炼平台在以下方面更适合国内企业与开发者使用

<img src="https://img.alicdn.com/imgextra/i4/O1CN01lbXhaX1WpOeXxkHwi_!!6000000002837-2-tps-1672-944.png" alt="image" width="700">

百炼平台的多智能体编排功能，依托大模型的强大能力，支持智能体自主决策和任务分工。通过平台内置的画布式编排工具，用户可以轻松实现以下目标：
- 定义智能体的执行逻辑：用户能够直观地通过画布式工具配置各个智能体的执行规则和逻辑链路。
- 编排多个智能体之间的协作：通过模块化设计，各个智能体能够高效配合，完成复杂任务。
- 快速验证业务效果：内置的仿真和测试功能，帮助用户快速验证编排逻辑是否符合预期。

百炼平台支持用户从零开始设计智能体系统，即使没有深厚的技术背景，用户也可以通过其易用的界面快速搭建并验证多智能体协作系统。对于需要快速落地多智能体技术的企业，百炼平台是一个理想的选择。

详情请见[百炼智能体应用](https://help.aliyun.com/zh/model-studio/user-guide/single-agent-application)。

## 6.本章小结
在本节课程中，你学习了以下内容：
- Agent系统的核心理念
- 多智能体系统的构建原理和实现方法
- 大模型平台的多智能体编排


### 扩展阅读

你已经了解了如何使用多智能体系统来处理复杂任务，但这种设计不仅仅局限于基础操作。事实上，合理设计多智能体系统，能够帮助你将更为复杂的业务流程自动化处理，提升效率和精度，甚至可以实现一些需要人介入的繁琐操作。

##### 效果展示与流程图解析
下面是一个实际的智能体系统应用案例，假设你需要对一个网页进行内容验证——不仅仅是检查静态内容，还包括与页面元素的交互。此时，多智能体系统的优势就显现出来了。通过分工合作，多个智能体能够共同高效完成任务，而你无需手动干预每个步骤。

<a href="https://gw.alicdn.com/imgextra/i1/O1CN01w0oXM41YyfuOvSLsR_!!6000000003128-2-tps-1567-668.png" target="_blank">
<img src="https://gw.alicdn.com/imgextra/i1/O1CN01w0oXM41YyfuOvSLsR_!!6000000003128-2-tps-1567-668.png" alt="image" width="700">
</a>

当面对一个需要解析HTML页面并执行特定操作的任务时，多智能体的分工如下：

- Planner Agent规划器：分解任务，例如识别HTML元素中的列表或按钮。
- Selector Agent执行器：负责具体的操作任务，例如选择特定元素并执行点击动作。
- Monitor Agent监视器：实时监控任务的执行，确保流程按计划完成，如检测是否点击正确的按钮。

##### 视觉标注在智能体应用中的结合
在实际项目中，尤其是涉及到网页内容交互时，光靠传统的文本分析能力往往无法精确地识别界面元素，导致智能体的执行效率和准确性下降。为了解决这一问题，视觉标注被应用于自动化测试和控制台界面的元素识别。

在某些场景下，智能体在操作控制台界面时，由于界面元素的复杂性，模型可能难以准确地识别目标。此时，通过引入视觉标注技术，可以提高模型对界面元素的理解和操作准确性。

就像你在使用一款软件时，可能需要通过眼睛去识别按钮、下拉菜单或表单，才能做出正确的操作。如果没有清晰的视觉标识，你可能会找错位置或操作错误。类似地，在智能体执行任务时，界面元素的准确识别至关重要。通过视觉标注，系统可以为这些元素加上“标签”，帮助智能体更准确地“看到”并正确执行操作。

例如，在处理HTML页面的自动化测试时，通过使用视觉标注，系统可以对页面中的按钮、链接或表单进行标记，并指导智能体精确选择和操作这些元素。这种视觉增强的设计大大提高了模型处理复杂界面的能力，使得多智能体系统在面对不同的UI元素时更加灵活高效。

##### 多智能体系统的灵活性与技术结合
通过这个案例，你可以发现多智能体系统的设计并不局限于某一类技术或场景。虽然在这个示例中多智能体系统结合了视觉标注，但实际上，多智能体系统的灵活性使得它能够与各种技术结合，解决不同的需求。

例如，你可以结合机器学习技术优化智能体的决策过程，或者使用图像处理技术提升界面元素的识别能力。

又如，你可以将多智能体系统与自然语言处理结合，使其能够理解和生成更复杂的语言指令，应用于智能客服或文本分析场景。

你可以根据需求自由选择多智能体系统的架构和技术组合，最终为你带来最佳的自动化解决方案。

多智能体系统不仅是一个工具，它更是一种思维方式，能够根据不同任务和场景灵活设计和调整，从而在不同的应用中发挥巨大价值。在今后的应用中，无论是面对页面自动化测试，还是处理更复杂的业务流程，合理设计的多智能体系统都能帮助你提高效率，优化操作，甚至替代部分人类工作，解放你的生产力。

## 🔥 7.课后小测验

### 🔍 单选题
<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>在本教程中，Planner Agent的作用是什么❓</b>

- A. 负责执行工具函数
- B. 负责总结Agent的输出内容
- C. 负责分析用户输入，并将任务分发到其它Agent上
- D. 负责打印关键日志

**【点击查看答案】**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **参考答案：C**  
📝 **解析**：  
- 在本教程中，Planner Agent 的主要功能是作为任务的分配者。它分析用户的输入内容，并根据具体任务需求，将不同的任务分配给最适合的其他 Agent。
- 因此，Planner Agent在系统中起到路由的作用，确保每个任务能够高效地交由相应的 Agent 处理。

</div>
</details>

